Petit notebook juste pour montrer la rapidité du module numba.
Je vais utiliser le décorateur njit qui permet d'appliquer la compilation numba en étant sûr que la fonction ne soit pas exécutée en python.

In [1]:
import numpy as np
from numba import njit

Fonction qui crée une liste de n nombres ayant leur indice au carré comme valeur puis calcule la somme totale

In [2]:
def append_list(nombre: int) -> int:
    l: list[int] = []
    for i in range(nombre):
        l.append(i**2)
    return sum(l)

In [3]:
nombres = int(1e6)

In [4]:
%timeit append_list(nombres)

62.6 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Exactement la même fonction mais avec numba

In [5]:
@njit
def append_list_numba(nombre: int) -> int:
    l: list[int] = []
    for i in range(nombre):
        l.append(i**2)
    return sum(l)

# Petite particularité, le code est compilé donc la première exécution prend plus de temps, je l'appelle donc une première fois pour le compiler
append_list_numba(1)

0

In [6]:
%timeit append_list_numba(nombres)

8.14 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Quasiment 8 fois plus rapide en ne faisant rien.
Imaginons qu'on l'optimise un peu.

In [7]:
@njit
def append_list_numba_opt(nombre: int) -> int:
    # On crée directement un array de la bonne taille pour ne pas avoir à changer sa taille à chaque fois
    np_list: np.NDArray[int] = np.zeros(nombre)
    for i in range(nombre):
        np_list[i] = i**2
    return np.sum(np_list)

append_list_numba_opt(1)

0.0

In [8]:
%timeit append_list_numba_opt(nombres)

2.79 ms ± 91.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


~22 fois plus rapide que la fonction initiale

Et même si ici on peut se dire que c'est efficace parce-qu'on crée une liste et append à chaque fois, ça fonctionne aussi avec des fonctions beaucoup plus simples.
On va créer deux nouvelles fonctions réalisant le même calcul que précédemment mais sans passer par la création de liste.

In [9]:
def calcul_direct(nombre: int):
    somme = 0
    for i in range(nombre):
        somme += i**2
    return somme

In [10]:
nombres2 = int(1e6)

In [11]:
%timeit calcul_direct(nombres2)

37.4 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
@njit
def calcul_direct_numba(nombre: int):
    somme = 0
    for i in range(nombre):
        somme += i**2
    return somme

calcul_direct_numba(1)

0

In [13]:
%timeit calcul_direct_numba(nombres2)

476 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


~80 fois plus rapide ... alors qu'on fait juste une boucle ...